In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

In [3]:
def image_generator(directory, filenames, labels, batch_size=64, target_size=(224, 224), shuffle=True):
    num_samples = len(filenames)
    indices = np.arange(num_samples)
    if shuffle:
        np.random.shuffle(indices)

    while True:
        for start_idx in range(0, num_samples, batch_size):
            batch_indices = indices[start_idx:start_idx + batch_size]
            batch_images = []
            batch_labels = []
            for idx in batch_indices:
                filename = filenames[idx]
                label = labels[idx]
                img_path = os.path.join(directory, filename)
                img = image.load_img(img_path, target_size=target_size)
                img_array = image.img_to_array(img)
                img_array = img_array / 255.0  # Rescale to [0,1]
                batch_images.append(img_array)
                batch_labels.append(label)

            yield np.array(batch_images), np.array(batch_labels)

In [4]:
dataset_dir = "D:/yapayzekafinal/resim"  # Ana veri dizini
classes = os.listdir(dataset_dir)
num_classes = len(classes)
print(classes)

['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [5]:
def load_data():
    filenames = []
    labels = []
    for i, cls in enumerate(classes):
        class_dir = os.path.join(dataset_dir, cls)
        class_filenames = os.listdir(class_dir)
        class_filenames = [os.path.join(cls, filename) for filename in class_filenames]
        filenames.extend(class_filenames)
        labels.extend([i] * len(class_filenames))
    return filenames, labels

In [6]:
filenames, labels = load_data()
X_train_filenames, X_test_filenames, y_train, y_test = train_test_split(filenames, labels, test_size=0.2, random_state=42)

In [11]:
cnn_model = Sequential()

# Convolutional ve MaxPooling katmanları
cnn_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model.add(Conv2D(128, kernel_size=(3, 3), activation='sigmoid'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
# Tam bağlantılı (fully connected) katmanlar
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(64, activation='sigmoid'))

# Çıkış katmanı
cnn_model.add(Dense(10, activation='softmax'))  # 10 sınıf için çıkış

#model = Sequential([
#    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
#    MaxPooling2D((2, 2)),
#    Conv2D(64, (3, 3), activation='relu'),
#    MaxPooling2D((2, 2)),
#    Conv2D(128, (3, 3), activation='relu'),
#    MaxPooling2D((2, 2)),
#    Flatten(),
#    Dense(128, activation='relu'),
#    Dense(7, activation='softmax')
#])


In [12]:
cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
train_generator = image_generator(dataset_dir, X_train_filenames, y_train)
validation_generator = image_generator(dataset_dir, X_test_filenames, y_test)


In [14]:
cnn_model.fit(train_generator,
          steps_per_epoch=len(X_train_filenames) // 32,  # Eğitim veri setinizdeki toplam örnek sayısını uygun şekilde ayarlayın
          epochs=10,
          validation_data=validation_generator,
          validation_steps=len(X_test_filenames) // 32)

Epoch 1/10
  8/717 ━━━━━━━━━━━━━━━━━━━━ 1:33:57 8s/step - accuracy: 0.1261 - loss: 2.4377

KeyboardInterrupt: 

In [30]:
cnn_model.save("deneme2.keras")